In [ ]:
import cvxpy as cp
import numpy as np
import math
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from SOC_ACOPF_single_step import *
import gurobipy

In [2]:
# Case 33
# Base value
Sb = 10e6                 # VA
BaseMVA = Sb*1e-6         # MVA
Vb = 12.66e3              # V
Zb = Vb**2/Sb             # O
Ib = Sb*np.sqrt(3)/Vb     # A
N_Bus = 33
NT = 24
Cgrid = np.concatenate([208.7 * np.ones(7), 294.8 * np.ones(14), 208.7 * np.ones(3)])     # $/(p.u)/10^7

In [3]:
# Line
# No.(1)|From(2)|To(3)|Pd(kW)(4)|Qd(5)|r(6)(ohms)|x(7)|rpu(8)|xpu(9)|R/X(10)|Pup(11)
linedata = np.array([
    [1, 1, 2, 100, 60, 0.0922, 0.047, 0.0058, 0.0029, 2, 9900],
    [2, 2, 3, 90, 40, 0.493, 0.2511, 0.0308, 0.0157, 1.9618, 9900],
    [3, 3, 4, 120, 80, 0.366, 0.1864, 0.0228, 0.0116, 1.9655, 9900],
    [4, 4, 5, 60, 30, 0.3811, 0.1941, 0.0238, 0.0121, 1.9669, 9900],
    [5, 5, 6, 60, 20, 0.819, 0.707, 0.0511, 0.0441, 1.1587, 9900],
    [6, 6, 7, 200, 100, 0.1872, 0.6188, 0.0117, 0.0386, 0.3031, 9900],
    [7, 7, 8, 200, 100, 1.7114, 1.2351, 0.1068, 0.0771, 1.3852, 9900],
    [8, 8, 9, 60, 20, 1.03, 0.74, 0.0643, 0.0462, 1.3918, 9900],
    [9, 9, 10, 60, 20, 1.044, 0.74, 0.0651, 0.0462, 1.4091, 9900],
    [10, 10, 11, 45, 30, 0.1966, 0.065, 0.0123, 0.0041, 3, 9900],
    [11, 11, 12, 60, 35, 0.3744, 0.1238, 0.0234, 0.0077, 3.039, 9900],
    [12, 12, 13, 60, 35, 1.468, 1.155, 0.0916, 0.0721, 1.2705, 9900],
    [13, 13, 14, 120, 80, 0.5416, 0.7129, 0.0338, 0.0445, 0.7596, 9900],
    [14, 14, 15, 60, 10, 0.591, 0.526, 0.0369, 0.0328, 1.125, 9900],
    [15, 15, 16, 60, 20, 0.7463, 0.545, 0.0466, 0.034, 1.3706, 9900],
    [16, 16, 17, 60, 20, 1.289, 1.721, 0.0804, 0.1074, 0.7486, 9900],
    [17, 17, 18, 90, 40, 0.732, 0.574, 0.0457, 0.0358, 1.2765, 9900],
    [18, 2, 19, 90, 40, 0.164, 0.1565, 0.0102, 0.0098, 1.0408, 9900],
    [19, 19, 20, 90, 40, 1.5042, 1.3554, 0.0939, 0.0846, 1.1099, 9900],
    [20, 20, 21, 90, 40, 0.4095, 0.4784, 0.0255, 0.0298, 0.8557, 9900],
    [21, 21, 22, 90, 40, 0.7089, 0.9373, 0.0442, 0.0585, 0.7556, 9900],
    [22, 3, 23, 90, 50, 0.4512, 0.3083, 0.0282, 0.0192, 1.4688, 9900],
    [23, 23, 24, 420, 200, 0.898, 0.7091, 0.056, 0.0442, 1.267, 9900],
    [24, 24, 25, 420, 200, 0.896, 0.7011, 0.0559, 0.0437, 1.2792, 9900],
    [25, 6, 26, 60, 25, 0.203, 0.1034, 0.0127, 0.0065, 1.9538, 9900],
    [26, 26, 27, 60, 25, 0.2842, 0.1447, 0.0177, 0.009, 1.9667, 9900],
    [27, 27, 28, 60, 20, 1.059, 0.9337, 0.0661, 0.0583, 1.1338, 9900],
    [28, 28, 29, 120, 70, 0.8042, 0.7006, 0.0502, 0.0437, 1.1487, 9900],
    [29, 29, 30, 200, 600, 0.5075, 0.2585, 0.0317, 0.0161, 1.9689, 9900],
    [30, 30, 31, 150, 70, 0.9744, 0.963, 0.0608, 0.0601, 1.0116, 9900],
    [31, 31, 32, 210, 100, 0.3105, 0.3619, 0.0194, 0.0226, 0.8584, 9900],
    [32, 32, 33, 60, 40, 0.341, 0.5302, 0.0213, 0.0331, 0.6435, 9900]
])

N_line = linedata.shape[0]
sending_end = linedata[:, 1]-1 
receiving_end = linedata[:, 2]-1 
r = linedata[:, 7] #/ Zb 
x = linedata[:, 8] #/ Zb 
b = np.zeros(N_line)

In [4]:
#Generators gas turbine
# Loc.(1)|P_min MW (2)|P_max MW (3)|Q_min MVar (4)|Q_max(5) MVar|a $/MW^2(6)|b $/MW (7) | alpha $/MW^2 (8) | beta $/MW (9)
GTdata = np.array([
    [7, 0, 2000 * 1e3, -1*1e6, 0.5*1e6, 0.12, 20, 5, 6],
    [21, 0, 1500 * 1e3, -1*1e6, 1e6, 0.09, 15, 4, 5]
])

IndGT = GTdata[:, 0].astype(int)-1  # Indices of gas turbines
Pg_GT_l = np.zeros((N_Bus, NT))  # Lower generation limits
Pg_GT_u = np.zeros((N_Bus, NT))  # Upper generation limits
Qg_GT_l = np.zeros((N_Bus, NT))  # Lower reactive power limits
Qg_GT_u = np.zeros((N_Bus, NT))  # Upper reactive power limits

# Assigning generation limits based on indices
Pg_GT_l[IndGT, :] = GTdata[:, 1].reshape(-1, 1) / Sb  # p.u. lower limits
Pg_GT_u[IndGT, :] = GTdata[:, 2].reshape(-1, 1) / Sb  # p.u. upper limits
Qg_GT_l[IndGT, :] = GTdata[:, 3].reshape(-1, 1) / Sb  # p.u. lower reactive limits
Qg_GT_u[IndGT, :] = GTdata[:, 4].reshape(-1, 1) / Sb  # p.u. upper reactive limits

# Maximum generation limits
Pg_GT_max = Pg_GT_u  # Maximum real power generation in p.u.
Qg_GT_max = Qg_GT_u  # Maximum reactive power generation in p.u.

# Coefficients for generation costs
ag = np.zeros(N_Bus)  # Coefficient a
bg = np.zeros(N_Bus)  # Coefficient b

# Assigning cost coefficients based on indices
ag[IndGT] = GTdata[:, 5]  # a coefficients
bg[IndGT] = GTdata[:, 6]  # b coefficients

In [5]:
# PV
# PV Gen  loc(1)|RatedPower(2)(MW)
PVdata = np.array([[17, 0],
                   [32, 0]])

# Extracting the indices for PV generation
IndGenPV = PVdata[:, 0].astype(int)-1  # Indices of PV generators

# Define PV_data for generation in p.u.
PV_data = np.array([
    [0, 0, 0, 0, 0, 0.008640, 0.015120, 0.025200, 0.065520, 0.069120, 
     0.068400, 0.068400, 0.064800, 0.050400, 0.037440, 0.021600, 
     0.018720, 0.007920, 0.007920, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0.017496, 0.030618, 0.051030, 0.132678, 0.139968, 
     0.138510, 0.138510, 0.131220, 0.102060, 0.075816, 0.043740, 
     0.037908, 0.016038, 0.016038, 0, 0, 0, 0, 0]
])

# Initialize lower and upper limits for PV generation
PV_l = np.zeros((N_Bus, NT))  # Lower limits
PV_u = np.zeros((N_Bus, NT))  # Upper limits

# Assign values to the arrays based on indices
PV_l[IndGenPV, :] = 0  # Lower limits remain 0
PV_u[IndGenPV, :] = PV_data  # Upper limits from PV_data in p.u.

In [6]:
# Define the ESSdata array as a NumPy array
ESSdata = np.array([
    [18, 1e5, 1e5, 1.2 * 1e5, 0],  # w
    [33, 0.8 * 1e5, 0.8 * 1e5, 1.2 * 1e5, 0]  # w
])

# Extracting the indices for Energy Storage System
IndESS = ESSdata[:, 0].astype(int)-1  # Indices of ESS locations, converted to integers

# Charging limits
ESS_cha_l = np.zeros((N_Bus, NT))  # Lower charging limits
ESS_cha_u = np.zeros((N_Bus, NT))  # Upper charging limits
ESS_cha_l[IndESS, :] = ESSdata[:, 4].reshape(-1, 1) / Sb  # Min charging limits
ESS_cha_u[IndESS, :] = ESSdata[:, 1].reshape(-1, 1) / Sb  # Max charging limits

# Discharging limits
ESS_dis_l = np.zeros((N_Bus, NT))  # Lower discharging limits
ESS_dis_u = np.zeros((N_Bus, NT))  # Upper discharging limits
ESS_dis_l[IndESS, :] = ESSdata[:, 4].reshape(-1, 1) / Sb  # Min discharging limits
ESS_dis_u[IndESS, :] = ESSdata[:, 2].reshape(-1, 1) / Sb  # Max discharging limits

# State of Charge (SoC)
ESS_soc0 = np.zeros((N_Bus))
ESS_soc0[IndESS] = 0.2 * 1e5 * np.ones((len(IndESS))) / Sb  # Initial state of charge
ESS_soc_l = np.zeros((N_Bus, NT))  # Lower SoC limits
ESS_soc_u = np.zeros((N_Bus, NT))  # Upper SoC limits
ESS_soc_l[IndESS, :] = ESSdata[:, 4].reshape(-1, 1) / Sb   # Min SoC limits
ESS_soc_u[IndESS, :] = ESSdata[:, 3].reshape(-1, 1) / Sb   # Max SoC limits

In [7]:
# Load
# Active and reactive load factors
Pd_ratio = np.concatenate(([0], linedata[:, 3])) / np.sum(linedata[:, 3])  # Active load factor
Qd_ratio = np.concatenate(([0], linedata[:, 4])) / np.sum(linedata[:, 4])  # Reactive load factor

Pd_Season = np.array([63, 62, 60, 58, 59, 65, 72, 85, 95, 99, 100, 99, 93, 92, 90, 88, 90, 92, 96, 98, 96, 90, 80, 70]) / 15 - 1.5  
Pd_Season *= 1e6  # Convert to W

Qd_Season = np.array([18, 16, 15, 14, 15.5, 15, 16, 17, 18, 19, 20, 20.5, 21, 20.5, 21, 19.5, 20, 20, 19.5, 19.5, 18.5, 18.5, 18, 18]) / 10  
Qd_Season *= 1e6  # Convert to Var

# Distribute loads to each node
Pd = Pd_ratio[:, np.newaxis] * Pd_Season[:NT]  # Active load distribution
Qd = Qd_ratio[:, np.newaxis] * Qd_Season[:NT]  # Reactive load distribution

# Convert to p.u.
Pd = Pd / Sb  # 24 hours active load in p.u.
Qd = Qd / Sb  # 24 hours reactive load in p.u.

In [8]:
# Voltage limits
U2_l = (0.9 ** 2) * np.ones((N_Bus, NT))  # Lower voltage limit in p.u.
U2_u = (1.1 ** 2) * np.ones((N_Bus, NT))  # Upper voltage limit in p.u.
U2_max = U2_u.copy()  # Maximum voltage, same as upper limit

In [9]:
#Slack Bus
Pg_slack_l = np.zeros((N_Bus, NT))  # Lower generation limits
Pg_slack_u = np.zeros((N_Bus, NT))  # Upper generation limits
Qg_slack_l = np.zeros((N_Bus, NT))  # Lower reactive power limits
Qg_slack_u = np.zeros((N_Bus, NT))  # Upper reactive power limits
Pg_slack_u[0,:]=0.3
Qg_slack_u[0,:]=0.5
Qg_slack_l[0,:]=-0.5

#ag[0]=0.1
bg[0]=10 #Linear grid price

U2_l[0,:]=1
U2_u[0,:]=1

In [10]:
"""G_n = np.zeros(N_Bus)
B_n = np.zeros(N_Bus)
c = np.zeros(N_Bus)
K_l = 4*np.ones(N_line)
# Initialize storage for results across time steps
cost_all = []
pn_all = []
qn_all = []
Vn_all = []
psl_all = []
qsl_all = []
pol_all = []
qol_all = []
Kol_all = []
theta_n_all = []
theta_l_all = []
ESS_cha_all = []
ESS_dis_all = [] 
ESS_soc_all = []

ESS_soc = ESS_soc_u[:,0]

# Loop over each time step to solve the ACOPF problem
for t in range(NT):
    # Extract data for the current time step t
    v_bound = np.sqrt(np.array([U2_l[:,t],U2_u[:,t]]))
    pn_bound = (np.array([Pg_GT_l[:,t],Pg_GT_u[:,t]])+np.array([PV_l[:,t],PV_u[:,t]])+np.array([Pg_slack_l[:,t],Pg_slack_u[:,t]]))
    qn_bound = (np.array([Qg_GT_l[:,t],Qg_GT_u[:,t]])+np.array([Qg_slack_l[:,t],Qg_slack_u[:,t]]))
    ESS_cha_bound = np.array([ESS_cha_l[:,t],ESS_cha_u[:,t]])
    ESS_dis_bound = np.array([ESS_dis_l[:,t],ESS_dis_u[:,t]])
    ESS_soc_bound = np.array([ESS_soc_l[:,t],ESS_soc_u[:,t]])
    # Assuming each of these parameters is a time-series array/list where `t` index corresponds to the time step
    cost, pn, qn, Vn, psl, qsl, pol, qol, Kol, theta_n, theta_l, ESS_cha, ESS_dis = SOC_ACOPF(
        BaseMVA,
        sending_end.astype(int),
        receiving_end.astype(int),
        r,
        x,
        b,
        Pd[:,t],        
        Qd[:,t],        
        pn_bound.T,
        qn_bound.T,
        v_bound.T,
        G_n,
        B_n,
        K_l,
        ag,
        bg,
        c,
        ESS_soc, ESS_cha_bound.T, ESS_dis_bound.T, ESS_soc_bound.T)
    
    ESS_soc = ESS_soc + ESS_cha - ESS_dis
    
    # Append results to storage lists
    cost_all.append(cost)
    pn_all.append(pn)
    qn_all.append(qn)
    Vn_all.append(Vn)
    psl_all.append(psl)
    qsl_all.append(qsl)
    pol_all.append(pol)
    qol_all.append(qol)
    Kol_all.append(Kol)
    theta_n_all.append(theta_n)
    theta_l_all.append(theta_l)
    ESS_cha_all.append(ESS_cha)  
    ESS_dis_all.append(ESS_dis)  
    ESS_soc_all.append(ESS_soc)"""

'G_n = np.zeros(N_Bus)\nB_n = np.zeros(N_Bus)\nc = np.zeros(N_Bus)\nK_l = 4*np.ones(N_line)\n# Initialize storage for results across time steps\ncost_all = []\npn_all = []\nqn_all = []\nVn_all = []\npsl_all = []\nqsl_all = []\npol_all = []\nqol_all = []\nKol_all = []\ntheta_n_all = []\ntheta_l_all = []\nESS_cha_all = []\nESS_dis_all = [] \nESS_soc_all = []\n\nESS_soc = ESS_soc_u[:,0]\n\n# Loop over each time step to solve the ACOPF problem\nfor t in range(NT):\n    # Extract data for the current time step t\n    v_bound = np.sqrt(np.array([U2_l[:,t],U2_u[:,t]]))\n    pn_bound = (np.array([Pg_GT_l[:,t],Pg_GT_u[:,t]])+np.array([PV_l[:,t],PV_u[:,t]])+np.array([Pg_slack_l[:,t],Pg_slack_u[:,t]]))\n    qn_bound = (np.array([Qg_GT_l[:,t],Qg_GT_u[:,t]])+np.array([Qg_slack_l[:,t],Qg_slack_u[:,t]]))\n    ESS_cha_bound = np.array([ESS_cha_l[:,t],ESS_cha_u[:,t]])\n    ESS_dis_bound = np.array([ESS_dis_l[:,t],ESS_dis_u[:,t]])\n    ESS_soc_bound = np.array([ESS_soc_l[:,t],ESS_soc_u[:,t]])\n    # Ass

In [11]:
"""test_all = []
for t in range(NT):
    test = np.where(ESS_dis_all[t]>=1e-4,ESS_dis_all[t],0)
    test_all.append(test)
test_all"""

'test_all = []\nfor t in range(NT):\n    test = np.where(ESS_dis_all[t]>=1e-4,ESS_dis_all[t],0)\n    test_all.append(test)\ntest_all'

## Copy the code to detect what doesn't work

In [13]:
num_nodes = N_Bus
num_lines = N_line

sending_node = sending_end.astype(int)
receiving_node = receiving_end.astype(int)

bg[IndESS] = 5

G_n = np.zeros(N_Bus)
B_n = np.zeros(N_Bus)
c = np.zeros(N_Bus)
K_l = 4*np.ones(N_line)

R_l = r
X_l = x
B_l = b

A_plus, A_minus = Incidence_matrices(N_Bus,N_line,sending_node,receiving_node)

# theta_n min and max
theta_n_min = -np.pi/2*np.ones(num_nodes)  # Minimum bus angle 
theta_n_max = np.pi/2*np.ones(num_nodes)  # Maximum bus angle

# theta_l min and max
theta_l_min = -np.pi/2*np.ones(num_lines)  # Minimum line angle (from relaxation assumption)
theta_l_max = np.pi/2*np.ones(num_lines)  # Maximum line angle (from relaxation assumption)

######################################################
"""Variables"""

p_n = cp.Variable((num_nodes,NT))  # Active power at node n
q_n = cp.Variable((num_nodes,NT))  # Reactive power at node n
V_n = cp.Variable((num_nodes,NT))  # Voltage magnitude squared at node n
theta_n = cp.Variable((num_nodes,NT))  # Voltage angles at node n

p_sl = cp.Variable((num_lines,NT))  # Active power at sending end of line l
q_sl = cp.Variable((num_lines,NT))  # Reactive power at sending end of line l
p_ol = cp.Variable((num_lines,NT))  # Active power losses on line l
q_ol = cp.Variable((num_lines,NT))  # Reactive power losses on line l
K_ol = cp.Variable((num_lines,NT))  # Branch Equivalent ampacity constraint on line l
theta_l = cp.Variable((num_lines,NT))  # Voltage angles at line l

ESS_cha = cp.Variable((num_nodes,NT)) 
ESS_dis = cp.Variable((num_nodes,NT))
ESS_soc = cp.Variable((num_nodes,NT))
binary_var_ESS = cp.Variable((num_nodes,NT),boolean=True)  # Binary variable

#z_l = cp.Variable(num_lines)    # z_l >= sqrt(p_sl^2 + q_sl^2)

In [28]:
""" Constraints"""
constraints = []
objective = 0

for time in range(NT):

    v_bound = np.sqrt(np.array([U2_l[:,time],U2_u[:,time]])).T
    pn_bound = (np.array([Pg_GT_l[:,time],Pg_GT_u[:,time]])+np.array([PV_l[:,time],PV_u[:,time]])+np.array([Pg_slack_l[:,time],Pg_slack_u[:,time]])).T
    qn_bound = (np.array([Qg_GT_l[:,time],Qg_GT_u[:,time]])+np.array([Qg_slack_l[:,time],Qg_slack_u[:,time]])).T
    ESS_cha_bound = np.array([ESS_cha_l[:,time],ESS_cha_u[:,time]]).T
    ESS_dis_bound = np.array([ESS_dis_l[:,time],ESS_dis_u[:,time]]).T
    ESS_soc_bound = np.array([ESS_soc_l[:,time],ESS_soc_u[:,time]]).T

    p_d = Pd[:,time]
    q_d = Qd[:,time]

    # Unfolding nodes data
    p_n_min = pn_bound[:,0]  # Minimum active power generation at each node
    p_n_max = pn_bound[:,1]  # Maximum active power generation at each node
    q_n_min = qn_bound[:,0]  # Minimum reactive power generation at each node
    q_n_max = qn_bound[:,1]  # Maximum reactive power generation at each node
    V_min = v_bound[:,0]**2  # Minimum voltage squared at each node
    V_max = v_bound[:,1]**2  # Maximum voltage squared at each node
    ESS_cha_min = ESS_cha_bound[:,0] # Minimum charging rate at each node 
    ESS_cha_max = ESS_cha_bound[:,1] # Maximum charging rate at each node
    ESS_dis_min = ESS_dis_bound[:,0] # Minimum discharging rate at each node 
    ESS_dis_max = ESS_dis_bound[:,1] # Maximum discharging rate at each node
    ESS_soc_min = ESS_soc_bound[:,0] # Minimum state of charge at each node 
    ESS_soc_max = ESS_soc_bound[:,1] # Maximum state of charge at each node

    # All constraints located on the buses
    for n in range(num_nodes): 

        # Voltage Magnitude bounds (1k):
        constraints.append(V_n[n,time] >= V_min[n])
        constraints.append(V_n[n,time] <= V_max[n])

        # Node angle bounds (1m):
        constraints.append(theta_n[n,time] >= theta_n_min[n])
        constraints.append(theta_n[n,time] <= theta_n_max[n])

        # Active power bounds (1n):
        constraints.append(p_n[n,time] >= p_n_min[n])
        constraints.append(p_n[n,time] <= p_n_max[n])

        # Reactive power bounds (1o):
        constraints.append(q_n[n,time] >= q_n_min[n])
        constraints.append(q_n[n,time] <= q_n_max[n])

        # ESS charging and discharging rate bounds :
        constraints.append(ESS_cha[n,time] >= ESS_cha_min[n])
        constraints.append(ESS_cha[n,time] <= ESS_cha_max[n])
        constraints.append(ESS_dis[n,time] >= ESS_dis_min[n])
        constraints.append(ESS_dis[n,time] <= ESS_dis_max[n])

        # ESS min and max SOC
        constraints.append(ESS_soc[n,time] <= ESS_soc_max[n])
        constraints.append(ESS_soc[n,time] >= ESS_soc_min[n])

        # Linking time steps with ESS
        if time != NT-1:
            constraints.append(ESS_soc[n,time+1] == ESS_soc[n,time] + ESS_cha[n,time] - ESS_dis[n,time])
        # Initialyzing ESS
        if time == 0:
            constraints.append(ESS_soc[n,time] == ESS_soc0[n])

        #Adding binary variable to constraint the battery to either charge or discharge
        constraints.append(ESS_cha[n,time] <= 1e5 * (1-binary_var_ESS[n,time]))
        constraints.append(ESS_dis[n,time] <= 1e5 * binary_var_ESS[n,time])

        # Active power balance (1b):
        #constraints.append(p_n[n] - p_d[n] == A_plus[n, :]@p_sl - A_minus[n, :]@p_ol + G_n[n]*V_n[n])
        constraints.append(p_n[n,time] + ESS_dis[n,time] - p_d[n] - ESS_cha[n,time] == A_plus[n, :]@p_sl[:,time] - A_minus[n, :]@p_ol[:,time] + G_n[n]*V_n[n,time])

        # Reactive power balance (1c):
        constraints.append(q_n[n,time] - q_d[n] == A_plus[n, :]@q_sl[:,time] - A_minus[n, :]@q_ol[:,time] - B_n[n]*V_n[n,time])



    #All contrsiants located on the lines
    for l in range(num_lines): 

        # Line angle bounds (1l):
        constraints.append(theta_l[l,time] >= theta_l_min[l])
        constraints.append(theta_l[l,time] <= theta_l_max[l])

        # Voltage drop constraint (1d):
        constraints.append(V_n[sending_node[l],time] - V_n[receiving_node[l],time] == 2*R_l[l]*p_sl[l,time] + 2*X_l[l]*q_sl[l,time] - R_l[l]*p_ol[l,time] - X_l[l]*q_ol[l,time])

        # Conic active and reactive power losses constraint (2b):
        constraints.append(K_ol[l,time] == (K_l[l] - V_n[sending_node[l],time]*B_l[l]**2 + 2*q_sl[l,time]*B_l[l])*X_l[l])
        constraints.append(K_ol[l,time] >= q_ol[l,time])
        #constraints.append(0 <= p_ol[l])
        #constraints.append(cp.SOC(z_l[l], np.sqrt(X_l[l]) * cp.hstack([p_sl[l], q_sl[l]])))
        constraints.append(
            cp.norm(
                cp.vstack([
                    2 *np.sqrt(X_l[l])* cp.vstack([p_sl[l,time], q_sl[l,time]]),
                    cp.reshape(q_ol[l,time] - V_n[sending_node[l],time], (1, 1))
                ]),
                2
            ) <= q_ol[l,time] + V_n[sending_node[l],time]
        )

        # Power loss constraint (2c):
        constraints.append(p_ol[l,time] * X_l[l] == q_ol[l,time] * R_l[l])

        # Line angle constraint (1h):
        constraints.append(theta_l[l,time] == theta_n[sending_node[l],time] - theta_n[receiving_node[l],time])

        # Linearized angle constraint (2d):
        constraints.append(theta_l[l,time] == X_l[l]*p_sl[l,time] - R_l[l]*q_sl[l,time])

        # Feasibility solution recovery equation (4g):
        constraints.append(V_n[sending_node[l],time] + V_n[receiving_node[l],time] >= cp.norm(cp.vstack([2*theta_l[l,time]/np.sin(theta_l_max[l]), V_n[sending_node[l],time] - V_n[receiving_node[l],time]]),2))


    #####################################################################
    """Objective Function""" 
    # Minimize total generation cost by using quadratic relationship

objective = cp.Minimize(cp.sum(cp.multiply(ag.reshape((N_Bus, 1)), cp.square(p_n * BaseMVA)) + cp.multiply(bg.reshape((N_Bus, 1)), p_n * BaseMVA) + c))

In [29]:
# Defining the optimization problem
problem = cp.Problem(objective, constraints)
#####################################################################

#####################################################################
# Solve the problem
problem.solve(
    solver=cp.GUROBI,
    #max_iters=500000,
    #BarHomogeneous=True,
    #DualReductions = 0,
    #reoptimize=True,
    #NumericFocus=3,
    #QCPDual=1,
    verbose=True
)
print("Problem Status:", problem.status)
print("Optimal Value of the Objective Function:", problem.value)


                                     CVXPY                                     
                                     v1.5.3                                    
(CVXPY) Nov 12 12:23:14 PM: Your problem has 10944 variables, 22728 constraints, and 0 parameters.
(CVXPY) Nov 12 12:23:16 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 12 12:23:16 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 12 12:23:16 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 12 12:23:16 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Nov 12 12:23:18 PM: Compiling problem (target solver=GURO

In [30]:
print(np.sum(Pd))
print(np.sum(p_n.value))
#np.round((p_n.value+ESS_dis.value)*Sb)/Sb

9.68
9.958401018542919


In [30]:
K_ol.value
q_ol.value
test_mult = []
for t in range(N_line):
    test_mult.append(X_l[t]*(p_sl[t].value**2+q_sl[t].value**2)/V_n[sending_node[t]].value)
np.array(test_mult)

array([2.54404893e-04, 1.17302903e-03, 4.11406782e-04, 3.81689913e-04,
       1.31318466e-03, 2.63734482e-04, 4.36986002e-04, 1.55687011e-04,
       1.30878672e-04, 9.59834408e-06, 1.50314444e-05, 1.08378117e-04,
       4.94695844e-05, 1.61954558e-05, 1.05216798e-05, 1.72258901e-05,
       2.14377843e-06, 2.78940634e-06, 1.11252594e-05, 2.50813095e-06,
       3.09408861e-06, 1.16014658e-04, 2.16685096e-04, 5.36920763e-05,
       7.22754798e-05, 9.16982292e-05, 5.42520247e-04, 3.72519542e-04,
       1.08817361e-04, 8.10302167e-05, 1.28128261e-05, 1.07089673e-06])

In [34]:
ESS_soc.value
print_ess = np.where(ESS_soc.value < 1e-4, 0, ESS_soc.value)

# Display the result with decimal formatting
np.set_printoptions(suppress=True, precision=6)
print(print_ess[IndESS,:])

[[0.002    0.000228 0.       0.       0.006154 0.006154 0.012    0.012
  0.002    0.005494 0.005439 0.003685 0.003306 0.012    0.012    0.012
  0.012    0.012    0.012    0.012    0.004822 0.       0.       0.      ]
 [0.002    0.       0.       0.       0.       0.       0.005369 0.012
  0.004    0.008307 0.008303 0.004855 0.00403  0.012    0.012    0.012
  0.012    0.012    0.012    0.012    0.004122 0.       0.       0.      ]]


In [38]:
print(ESS_cha.value[IndESS])
#print(ESS_cha_u[IndESS])
print(np.where(ESS_dis.value < 1e-4, 0, ESS_dis.value)[IndESS])

[[0.       0.       0.       0.006121 0.       0.005846 0.       0.
  0.003494 0.       0.       0.       0.008694 0.       0.       0.
  0.       0.       0.       0.       0.       0.       0.       0.      ]
 [0.       0.       0.       0.       0.       0.005369 0.006631 0.
  0.004307 0.       0.       0.       0.00797  0.       0.       0.
  0.       0.       0.       0.       0.       0.       0.       0.      ]]
[[0.001772 0.000195 0.       0.       0.       0.       0.       0.01
  0.       0.       0.001754 0.000378 0.       0.       0.       0.
  0.       0.       0.       0.007178 0.004815 0.       0.       0.01    ]
 [0.001969 0.       0.       0.       0.       0.       0.       0.008
  0.       0.       0.003448 0.000825 0.       0.       0.       0.
  0.       0.       0.       0.007877 0.004122 0.       0.       0.008   ]]


In [ ]:
G_n = np.zeros(N_Bus)
B_n = np.zeros(N_Bus)
c = np.zeros(N_Bus)
K_l = 4*np.ones(N_line)
# Initialize storage for results across time steps
cost_all = []
pn_all = []
qn_all = []
Vn_all = []
psl_all = []
qsl_all = []
pol_all = []
qol_all = []
Kol_all = []
theta_n_all = []
theta_l_all = []

# Loop over each time step to solve the ACOPF problem
for t in range(NT):
    print(t)
    # Extract data for the current time step `t`
    v_bound = np.sqrt(np.array([U2_l[:,t],U2_u[:,t]]))
    pn_bound = (np.array([Pg_GT_l[:,t],Pg_GT_u[:,t]])+np.array([PV_l[:,t],PV_u[:,t]])+np.array([Pg_slack_l[:,t],Pg_slack_u[:,t]]))
    qn_bound = (np.array([Qg_GT_l[:,t],Qg_GT_u[:,t]])+np.array([Qg_slack_l[:,t],Qg_slack_u[:,t]]))
    # Assuming each of these parameters is a time-series array/list where `t` index corresponds to the time step
    cost, pn, qn, Vn, psl, qsl, pol, qol, Kol, theta_n, theta_l = SOC_ACOPF(
        BaseMVA,
        sending_end.astype(int),
        receiving_end.astype(int),
        r,
        x,
        b,
        Pd[:,t],        # Active power demand at time step `t`
        Qd[:,t],        # Reactive power demand at time step `t`
        pn_bound.T,
        qn_bound.T,
        v_bound.T,
        G_n,
        B_n,
        K_l,
        ag,
        bg,
        c)
    
    # Append results to storage lists
    cost_all.append(cost)
    pn_all.append(pn)
    qn_all.append(qn)
    Vn_all.append(Vn)
    psl_all.append(psl)
    qsl_all.append(qsl)
    pol_all.append(pol)
    qol_all.append(qol)
    Kol_all.append(Kol)
    theta_n_all.append(theta_n)
    theta_l_all.append(theta_l)

0
Problem Status: optimal
Optimal Value of the Objective Function: 28.423655628458896
1
Problem Status: optimal
Optimal Value of the Objective Function: 27.604646203741673
2
Problem Status: optimal
Optimal Value of the Objective Function: 26.125596477902384
3
Problem Status: optimal
Optimal Value of the Objective Function: 24.656569473828082
4
Problem Status: optimal
Optimal Value of the Objective Function: 25.430396718634846
5
Problem Status: optimal
Optimal Value of the Objective Function: 26.432822109036834
6
Problem Status: optimal
Optimal Value of the Objective Function: 29.251521325675416
7
Problem Status: optimal
Optimal Value of the Objective Function: 37.69252197421288
8
Problem Status: optimal
Optimal Value of the Objective Function: 29.330866736305897
9
Problem Status: optimal
Optimal Value of the Objective Function: 31.499358341406275
10
Problem Status: optimal
Optimal Value of the Objective Function: 32.914914871658425
11
Problem Status: optimal
Optimal Value of the Object

In [55]:
(np.array([Pg_GT_l,Pg_GT_u])+np.array([PV_l,PV_u])+np.array([Pg_slack_l,Pg_slack_u]))[1][:,2]

array([0.3 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.2 , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.15, 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ])

In [58]:
(np.array([Pg_GT_l[:,time],Pg_GT_u[:,time]])+np.array([PV_l[:,time],PV_u[:,time]])+np.array([Pg_slack_l[:,time],Pg_slack_u[:,time]])).T[:,1]

array([0.3 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.2 , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.15, 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ])

In [33]:
for t in range(24):
    pn_bound = np.array([Pg_GT_l[:,t],Pg_GT_u[:,t]])+np.array([PV_l[:,t],PV_u[:,t]])
    print("sum demand: ",np.sum(Pd[:,t]))
    print("sum production max: ",np.sum(pn_bound.T[:,1]),"\n")
print("tot demand",np.sum(Pd))
print("tot prod", np.sum(Pg_GT_u+PV_u))

sum demand:  0.26999999999999996
sum production max:  0.35 

sum demand:  0.26333333333333336
sum production max:  0.35 

sum demand:  0.25000000000000006
sum production max:  0.35 

sum demand:  0.23666666666666666
sum production max:  0.35 

sum demand:  0.2433333333333333
sum production max:  0.35 

sum demand:  0.28333333333333327
sum production max:  0.376136 

sum demand:  0.32999999999999996
sum production max:  0.39573800000000003 

sum demand:  0.4166666666666667
sum production max:  0.42623 

sum demand:  0.4833333333333333
sum production max:  0.5481980000000001 

sum demand:  0.51
sum production max:  0.559088 

sum demand:  0.5166666666666667
sum production max:  0.55691 

sum demand:  0.51
sum production max:  0.55691 

sum demand:  0.47000000000000003
sum production max:  0.54602 

sum demand:  0.46333333333333343
sum production max:  0.50246 

sum demand:  0.45
sum production max:  0.463256 

sum demand:  0.4366666666666666
sum production max:  0.41534 

sum demand:  0.

In [32]:
np.sum(pn_bound)

0.35